In [1]:
%gui qt
%matplotlib qt
import sys, os
sys.path.append("..")
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
from trimesh import triangles as triangles_mod
from trimesh.ray.ray_triangle import ray_triangle_id
import mpl_toolkits.mplot3d as mpl3
from mpl_toolkits.mplot3d import Axes3D

Matplotlib is building the font cache; this may take a moment.


In [2]:
folder_path = r"D:\Magnumfe\al6-1-1_peem_profile2"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
raytracing_file = os.path.join(data_folder, "raytracing.npy")

indx = [int(mf.split('_')[-1].split('.')[0]) for mf in mag_files]
mag_files = [mf for _, mf in sorted(zip(indx, mag_files))]

In [3]:
# load raytracing if exists
raytr = np.load(raytracing_file, allow_pickle=True).item()
struct = raytr.struct
struct_projected = raytr.struct_projected
msh = raytr.mesh

In [4]:
ray_origins = struct_projected.triangles_center

p = get_projection_vector(90, 16)
triangles = raytr.mesh.triangles

triangles_normal = triangles_mod.normals(triangles)[0]
tree = triangles_mod.bounds_tree(triangles)

pnew = p[np.newaxis, :]

In [5]:
def ray_piercing_fun(orig): return ray_triangle_id(
    triangles, orig[np.newaxis, :], pnew, triangles_normal=triangles_normal, tree=tree)

In [45]:
orig = ray_origins[100]
tri_id, _, locs = ray_piercing_fun(orig)
tet_id = tri_id // 4
i = 2
tet = msh.tetra[tet_id[i]]

intersected_tetrahedra_indx_unique = np.unique(tet_id)
locs_pierced = locs[tet_id == tet_id[i]]

In [49]:
vts = msh.points[tet]
fig = plt.figure()

ax = fig.gca(projection='3d', proj_type='ortho')
# ax = fig.add_subplot((111), projection='3d', proj_type='ortho')
# ax.plot_trisurf(vts[:,0], vts[:,1], vts[:,2])


vert_idx = [[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]]
poly3d = [[vts[vert_idx[ix][iy]] for iy in range(len(vert_idx[0]))] for ix in range(len(vert_idx))]


tri = mpl3.art3d.Poly3DCollection(poly3d, linewidths=0)
ax.add_collection3d(mpl3.art3d.Line3DCollection(poly3d, colors='k', linewidths=2))

ax.set_xlim([vts[:,0].min(), vts[:, 0].max()])
ax.set_ylim([vts[:,1].min(), vts[:, 1].max()])
ax.set_zlim([vts[:,2].min(), vts[:, 2].max()])
tri.set_alpha(0.3)
tri.set_color('grey')
ax.add_collection3d(tri)

pt0 = orig
nx0 = np.cross(pt0, p)
# y = np.linspace(vts[:,1].min(), vts[:,1].max(), 100)

# xlim, ylim, zlim = [-190, -180], [500, 507], [498, 508]
# # ax.set_xticks([-500, 0, 500])
# # ax.set_yticks([-500, 0, 500])
# ax.set_xlim3d(xlim)
# ax.set_ylim3d(ylim)
# ax.set_zlim3d(zlim)
# ax.set_box_aspect((1, (ylim[1] - ylim[0])/(xlim[1] - xlim[0]), (zlim[1] - zlim[0])/(xlim[1] - xlim[0])))

y = np.linspace(ylim[0], ylim[1], 100)
x = (y*p[0]+nx0[2])/p[1]
z = (y*p[2]-nx0[0])/p[1]

ax.plot(x, y, z, c='#FF61FF', linewidth=2)
ax.plot(locs_pierced[:,0], locs_pierced[:,1], locs_pierced[:,2], c='tab:red', linewidth=3, marker='o', markersize=5)
ax.set_xlabel('x [nm]')
ax.set_ylabel('y [nm]')#, labelpad=20)
ax.set_zlabel('z [nm]')#, labelpad=55)
ax.tick_params(axis='z', which='major')#, pad=30)
ax.tick_params(axis='y', which='major')#, pad=10)
plt.savefig('tet_piercing.svg')

In [12]:
[vts[:,0].min(), vts[:, 0].max()]

[-188.1557581221328, -183.5904760652104]

In [13]:
[vts[:,1].min(), vts[:, 1].max()]

[501.6798051028813, 506.9517431773201]